In [1]:
import cv2
import rosbag
import numpy as np
import matplotlib.pyplot as plt
import sensor_msgs.point_cloud2 as pc2

In [2]:
checkerboard = []

with rosbag.Bag("./ros-simulations/data_collection/3d/1/1.bag", "r") as bag:
    for topic, msg, t in bag.read_messages(topics=['/3d_lidar/depth/points']):
        pc_data = pc2.read_points(msg, field_names=("x", "y", "z", "rgb"), skip_nans=True)

        points = []
        points.extend(pc_data)

        points = np.array(points)

        points_y = points[:, 1]
        points_y = np.round(points_y, 3)
        mask = (points_y < 0.07)

        checkerboard = points[:, [0, 1]][mask]
        break

print(f"Shape of the checkerboard points: {checkerboard.shape}")

x_len, y_len = np.max(checkerboard, axis=0) - np.min(checkerboard, axis=0)
print(f"Length of the checkerboard: {x_len:.2f} x {y_len:.2f}")

Shape of the checkerboard points: (2620, 2)
Length of the checkerboard: 0.20 x 0.20


In [3]:
x_corner_coords, y_corner_coords = np.linspace(np.min(checkerboard, axis=0), np.max(checkerboard, axis=0), num=9, axis=0).T

x_corner_coords = x_corner_coords[1:-1]
y_corner_coords = y_corner_coords[1:-1]

lidar_correspondences = np.array(np.meshgrid(x_corner_coords, y_corner_coords)).T.reshape(-1, 2)
print(f"Shape of the lidar correspondences: {lidar_correspondences.shape}")

Shape of the lidar correspondences: (49, 2)


In [4]:
K = np.array([
    1345.61962890625, 0.0,              960.0, 
    0.0,              1345.61962890625, 540.0,
    0.0,              0.0,              1.0
]).reshape(3, 3)